In [22]:
!pip -q install openai

In [23]:
import openai
import json

In [24]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    """This is a stub method to return fixed weather. This is where you can call API such as weather.com to fetch the weather"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [25]:
def summarize_forecast(message, function_name, function_response, city):
    """Take the orginal question plus forcast and let GTP summarize it into a natural lanauge."""
    summary = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": f"What is the weather like in {city}?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
    return summary["choices"][0]["message"]

In [26]:
city = "Dallas"

def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": f"What is the weather like in {city}?"}],
        functions=[
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        ],
        function_call="auto",
    )

    message = response["choices"][0]["message"]

    # Check if GPT response has funtion call. 
    # If so, execute the funtion
    if message.get("function_call"):
        function_name = message["function_call"]["name"]

        if function_name == "get_current_weather": 
            function_response = get_current_weather(
                location=message.get("location"),
                unit=message.get("unit"),
            )

            summary = summarize_forecast(message, function_name, function_response, city)

            return summary

print(run_conversation())

{
  "role": "assistant",
  "content": "The current weather in Dallas, TX is 72\u00b0F and sunny with windy conditions."
}
